# songs 불러와 확인

In [33]:
import pandas as pd
songs = pd.read_csv('songs_fin_mod.csv')
songs.head()

,genre,song_id,artist_id,song_name,artist_name,lyric
0,포크,31263577,468244,매트리스,10CM,오늘밤 너는 나와 이불 속에 들어가 아무것도 하지 말고 그냥 바라보다가 웃음을 참지...
1,포크,8194007,968452,오늘,오왠 (O.WHEN),새벽4시 잠들지 않아 돌아갈 수 없는 시간들을 생각하곤 해 습관처럼 마음이 아려...
2,포크,30657311,792022,나의 사춘기에게,볼빨간사춘기,나는 한때 내가 이 세상에 사라지길 바랬어 온 세상이 너무나 캄캄해 매일 밤을 울던...
3,포크,30611680,468244,폰서트,10CM,이건 세상에서 제일 비싼 단독 공연 가수는 나고 관객은 너 하나 화려한 막이 이제 ...
4,포크,9620473,792022,나만 안되는 연애,볼빨간사춘기,왠지 오늘따라 마음이 아픈지 했더니 오늘은 그대가 날 떠나가는 날이래요 왜 항상 나...


In [34]:
songs.shape

(2637, 6)

In [35]:
# genre별 갯수 확인 - 6:4 비율 맞는지 체크
songs.genre.value_counts()

발라드    559
힙합     456
댄스     447
트로트    435
포크     432
국악     308
Name: genre, dtype: int64

# 텍스트(가사) 데이터 정제

In [36]:
import re

##### 3000여곡의 각 가사들에 대해 전처리를 해야 하므로 함수 만들어두기

In [37]:
def preprocess(text):
  # 한글, 영어, 숫자만 남기기
  text = re.sub(r'[^a-zA-Z0-9가-힣]', ' ', text)
  # 영문을 소문자로
  text = text.lower()
  # 두 칸 이상 공백을 한 칸으로
  text = re.sub(r'\s+', ' ', text)
  return text

In [38]:
songs['lyric'] = songs['lyric'].apply(preprocess)
songs.head()
# 저장하기
songs.to_csv('songs_preprocessed.csv', index=False)

# 가사 토큰화

In [39]:
import konlpy
from konlpy.tag import Okt
okt = Okt()

In [40]:
def tokenize(text):
  return okt.morphs(text)

# 로지스틱 회귀모델을 활용한 분류

In [41]:
# import train-test-split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(songs['lyric'], songs['genre'], test_size=0.2, random_state=42)

In [42]:
X_train.shape

(2109,)

### 최적의 매개변수 조합을 찾기 위해 5-fold cross validation을 사용하기

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=str.split, preprocessor=None, lowercase=False, stop_words=None, max_df=0.7) 
#tokenizer=tokenize 대신 str.split으로 하면 더 빠른 대신 결과가 부정확 > 매개변수 조합을 찾는 것이니까 이렇게 해도 됨. 안 그러면 1시간 걸림
param_grid = [{'vect__ngram_range': [(1, 1)],
                'vect__tokenizer': [str.split],
                'clf__penalty': ('l1', 'l2'),
                'clf__C': (0.1, 1, 10, 100, 1000)}]

lr_tfidf = Pipeline([('vect', tfidf),
                      ('clf', LogisticRegression(solver='liblinear', max_iter=1000, random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(tokenizer= tokenize , preprocessor=None, lowercase=False, stop_words=None, max_df=0.7) 
# param_grid = [{'vect__ngram_range': [(1, 1)],
#                 'vect__tokenizer': [tokenize],
#                 'clf__penalty': ('l1', 'l2'),
#                 'clf__C': (0.1, 1, 10, 100, 1000)}]

# lr_tfidf = Pipeline([('vect', tfidf),
#                       ('clf', LogisticRegression(solver='liblinear', max_iter=1000, random_state=0))])

# gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1)
# gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False,
                                                        max_df=0.7,
                                                        tokenizer=<function tokenize at 0x7f9ec3396cb0>)),
                                       ('clf',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid=[{'clf__C': (0.1, 1, 10, 100, 1000),
                          'clf__penalty': ('l1', 'l2'),
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenize at 0x7f9ec3396cb0>]}],
             scoring='accuracy', verbose=1)

In [ ]:
# 최적의 매개변수 조합 print
print('최적의 매개변수 조합:', gs_lr_tfidf.best_params_)
# l2 규제 사용, 규제값은 10.0 사용

최적의 매개변수 조합: {'clf__C': 10, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <method 'split' of 'str' objects>}


In [45]:
# cv 정확도 출력
print('최종 정확도:', gs_lr_tfidf.best_score_)
# test 정확도 출력
y_pred = gs_lr_tfidf.predict(X_test)
clf = gs_lr_tfidf.best_estimator_
print('test 정확도:', clf.score(X_test, y_test))

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [44]:
from sklearn.metrics import confusion_matrix
# confusion matrix
labels = ['발라드', '힙합', '댄스', '트로트', '포크', '국악']
cm = confusion_matrix(y_test, y_pred, labels=labels)
print(cm)
import seaborn as sns
import matplotlib.pyplot as plt
# visualize confusion matrix
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt="d", linewidths=.5)
# label
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

plt.show()

# 4를 제일 못 맞춤, 0도 그닥

ValueError: Found input variables with inconsistent numbers of samples: [528, 1]